In [1]:
# Import necessary modules and functions
import os, datetime
import pandas as pd
from ETL.ETL_general import get_incremental_data, write_incremental_data, update_incremental, update_incremental_api, write_incremental_data
from ETL.ETL_garmin_api import init_garmin, get_garmin_data
from ETL.ETL_whoop import init_whoop, get_sleep_recovery_data, get_journal_data
from ETL.ETL_mfp_api import init_mfp, get_meal_data, get_meal_daily
from ETL.ETL_apple_health import get_food_time_data, get_weight_data
from ETL.ETL_libreview import get_glucose_daily, get_glucose_time
from ETL.ETL_trainingpeaks import get_tp_data

In [4]:
### Update data of intemediate clean files ###

# Apple health update (food times and weight)
apple_health_file_raw = 'Data/Apple health/exportacion.xml'
food_times_file = 'Data/Cleaned/Food.csv'
weight_file = 'Data/Cleaned/Weight.csv'
update_incremental(apple_health_file_raw, food_times_file, get_food_time_data)
update_incremental(apple_health_file_raw, weight_file, get_weight_data)

# Garmin update
garmin_file = 'Data/Cleaned/Garmin_daily.csv'
email_g = os.getenv("USERNAME_G")
password_g = os.getenv("PASSWORD_G")
garmin_client = init_garmin(email_g, password_g)
update_incremental_api(garmin_client, garmin_file, get_garmin_data)

# Glucose update
libreview_file_raw = 'Data/LibreLink/AlbertoRequena Izard_glucose.csv'
glucose_daily_file = 'Data/Cleaned/Glucose_daily.csv'
glucose_time_file = 'Data/Cleaned/Glucose.csv'
update_incremental(libreview_file_raw, glucose_daily_file, get_glucose_daily)

# MyFitnessPal API update
meals_file = 'Data/Cleaned/MFP meals scrapped.csv'
meals_daily_file = 'Data/Cleaned/MFP per day scrapped.csv'
mfp_client = init_mfp()
get_meal_data(mfp_client, meals_file)
get_meal_daily(mfp_client, meals_daily_file)

# Whoop API update
whoop_file = 'Data/Cleaned/Sleep_and_recovery.csv'
journal_file_raw = 'Data/Whoop/journal_entries.csv'
journal_file = 'Data/Cleaned/Journal.csv'
email_w = os.getenv("USERNAME_W")
password_w = os.getenv("PASSWORD_W")
whoop_client = init_whoop(email_w, password_w)
update_incremental_api(whoop_client, whoop_file, get_sleep_recovery_data)
get_journal_data(journal_file_raw, journal_file) # Not incremental for now

# Trainingpeaks update
tp_file_raw = 'Data/TrainingPeaks/workouts.csv'
tp_file = 'Data/Cleaned/TSS metrics.csv'
update_incremental(tp_file_raw, tp_file, get_tp_data)

print('Clean data files updated')

Data from 'Data/Apple health/exportacion.xml' is already up to date
Data from 'Data/Apple health/exportacion.xml' is already up to date
Logging in to Garmin Connect'
Data for 'Data/Cleaned/Garmin_daily.csv' is already up to date
Data from 'Data/LibreLink/AlbertoRequena Izard_glucose.csv' is already up to date
Data for 'Data/Cleaned/Sleep_and_recovery.csv' is already up to date
Journal data cleaned and saved to Data/Cleaned/Journal.csv
Data from 'Data/TrainingPeaks/workouts.csv' is already up to date
Clean data files updated
